In [ ]:
#Intel challenge for scene classification
#Challenge link: https://datahack.analyticsvidhya.com/contest/practice-problem-intel-scene-classification-challe/
#DataSet link: https://www.kaggle.com/nitishabharathi/scene-classification

In [1]:
#Load required packages 
import pandas as pd
import numpy as np
import glob
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os,shutil
from tensorflow.keras import models,layers,optimizers, Model
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [2]:
#Reading train data
df= pd.read_csv("train.csv")
df = df.replace(0,'buildings').replace(1,'forest').replace(2,'glacier').replace(3,'mountain').replace(4,'sea').replace(5,'street')

In [3]:
#Image Augmentation using keras ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255,
                           shear_range = 0.2,
                           zoom_range = 0.2,
                           horizontal_flip=True,
                           preprocessing_function= preprocess_input,
                           validation_split=0.25)
train_generator = datagen.flow_from_dataframe(dataframe=df,
                        directory="train/",
                        x_col="image_name",
                        y_col="label",
                        subset="training",
                        batch_size=100,
                        seed=42,
                        shuffle=True,
                        class_mode="categorical",
                        target_size=(150,150))
valid_generator = datagen.flow_from_dataframe(dataframe=df,
                        directory="train/",
                        x_col="image_name",
                        y_col="label",
                        subset="validation", batch_size=10,
                        seed=42,
                        shuffle=True,
                        class_mode="categorical",
                        target_size=(150,150))

Found 12776 validated image filenames belonging to 6 classes.
Found 4258 validated image filenames belonging to 6 classes.


In [6]:
#Model construction 
def conv_bn(x):
    x = layers.Conv2D(filters=64, kernel_size=3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    return layers.ReLU()(x)

inputs = layers.Input(shape=(150,150, 3))
x = conv_bn(inputs)
x = conv_bn(x)
x = conv_bn(x)
x = conv_bn(x)
x = conv_bn(x)
x = layers.Flatten()(x)
outputs = layers.Dense(6, activation="softmax")(x)
model = Model(inputs=inputs, outputs=outputs)
model.summary()
model.compile(loss='categorical_crossentropy',optimizer = optimizers.RMSprop(lr=0.01),metrics=['acc'])

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 75, 75, 64)        1792      
_________________________________________________________________
batch_normalization_9 (Batch (None, 75, 75, 64)        256       
_________________________________________________________________
re_lu_9 (ReLU)               (None, 75, 75, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 38, 38, 64)        36928     
_________________________________________________________________
batch_normalization_10 (Batc (None, 38, 38, 64)        256       
_________________________________________________________________
re_lu_10 (ReLU)              (None, 38, 38, 64)        0   

In [ ]:
#Train the model
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
history =model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=100,verbose=1)

In [1]:
from tensorflow.keras.models import load_model
model = load_model("RepConv.h5")
model.summary()

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 150, 150, 3)]     0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 75, 75, 64)        1792      
_________________________________________________________________
batch_normalization_31 (Batc (None, 75, 75, 64)        256       
_________________________________________________________________
re_lu_23 (ReLU)              (None, 75, 75, 64)        0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 38, 38, 64)        36928     
_________________________________________________________________
batch_normalization_32 (Batc (None, 38, 38, 64)        256       
_________________________________________________________________
re_lu_24 (ReLU)              (None, 38, 38, 64)        0   